In [37]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict
import string

import re


In [39]:
popular_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/popular.json")
assert len(popular_examples) == 1
popular_examples = popular_examples[0]

# random_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/random.json")
# assert len(random_examples) == 1
# random_examples = random_examples[0]

recent_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/recent.json")
assert len(recent_examples) == 1
recent_examples = [] # recent_examples[0]

ripple_edits_examples = popular_examples +  recent_examples # + random_examples
len(ripple_edits_examples)

885

In [63]:
# random_examples[0]

In [40]:
non_zero_outerloop_count = 0
strong_meta_examples = []
weak_meta_examples = []
for example in ripple_edits_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    # non_zero_outerloop_count += len(outerloop_instances) > 0
    if len(outerloop_instances) > 0 and len(locality_instances) > 0:
        
        outerloop_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        outerloop_queries = [q for q in outerloop_queries if len(q["answers"]) > 0]
        outerloop_queries = [q for q in outerloop_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in outerloop_queries])
        
        if len(outerloop_queries) == 0:
            continue
        
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        strong_meta_examples.append(example)
    elif len(locality_instances) == 0:
        
        outerloop_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        outerloop_queries = [q for q in outerloop_queries if len(q["answers"]) > 0]
        outerloop_queries = [q for q in outerloop_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in outerloop_queries])
        
        if len(outerloop_queries) == 0:
            continue
        
        weak_meta_examples.append(example)
    else:
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        weak_meta_examples.append(example)
        
print("# strong examples", len(strong_meta_examples))
print("# weak examples", len(weak_meta_examples))

# strong examples 762
# weak examples 115


In [41]:
len(strong_meta_examples) + len(weak_meta_examples)

877

In [47]:
80 / 877

0.09122006841505131

In [43]:
# [[a["value"] for a in q["answers"] if len(a["value"].strip()) > 0] for q in locality_queries]

In [ ]:
n_test = 80
n_valid = 80
np.random.shuffle(strong_meta_examples)
np.random.shuffle(weak_meta_examples)

test_examples = strong_meta_examples[:n_test]
valid_examples = strong_meta_examples[n_test:n_test+n_valid]
train_examples = strong_meta_examples[n_test+n_valid:] + weak_meta_examples
np.random.shuffle(train_examples)

In [ ]:
io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_popular/train.jsonl")
io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_popular/valid.jsonl")
io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_popular/test.jsonl")

In [38]:
len(train_examples)

1593

In [13]:
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/train.jsonl")

In [14]:
non_zero_outerloop_count = 0
meta_examples = []
# strong_meta_examples = []
# weak_meta_examples = []
count_rel = defaultdict(int)
count_prop_types = defaultdict(int)

for example in train_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    example["edit"]["text"] = example["edit"]["prompt"]
    meta_examples.append(example["edit"])
    
    outerloop_queries = []
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    
                    outerloop_queries.append(q)
    for outer_q in outerloop_queries:
        count_prop_types["efficacy::"+ outer_q["question_type"]] += 1
        count_rel["efficacy::"+ outer_q["relation"]] += 1
    # assert len(outerloop_queries) > 0
    meta_examples.extend(outerloop_queries)
    
    locality_queries = []
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    locality_queries.append(q)
    for loc_q in locality_queries:
        count_prop_types["specificity::"+ loc_q["question_type"]] += 1
        count_rel["specificity::"+ loc_q["relation"]] += 1
    
    meta_examples.extend(locality_queries)
    
    # assert len(locality_queries) > 0
        
# print("# strong examples", len(strong_meta_examples))
# print("# weak examples", len(weak_meta_examples))

In [23]:
ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/test_w_triplet.hf")

In [24]:
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/test.jsonl")

In [29]:
len(ds)
len(test_data)

150

In [35]:
new_test_data = []

count = 0
for i in range(len(test_data)):
    original_test = test_data[i]
    augmentation = ds[i-count]
    if original_test["edit"]["prompt"] == "The name of the composer of Klavierstücke  I–IV is Karlheinz Stockhausen.":
        # print("found")
        original_test["edit"]["subject"] = "name of the composer of Klavierst\u00fccke  I\u2013IV"
        original_test["edit"]["object"] = "Karlheinz Stockhausen"
        assert original_test["edit"]["subject"] in original_test["edit"]["prompt"]
        assert original_test["edit"]["object"] in original_test["edit"]["prompt"]
        new_test_data.append(original_test)
        count += 1
        continue
        # print(original_test)
        # print(augmentation)
    else:
        
        assert original_test["edit"]["prompt"] == augmentation["prompt"], f"{i}@@" + original_test["edit"]["prompt"] + "@@" + augmentation["prompt"]
        assert augmentation["subject"] in original_test["edit"]["prompt"]
        assert augmentation["object"] in original_test["edit"]["prompt"]

        original_test["edit"]["subject"] = augmentation["subject"]
        original_test["edit"]["object"] = augmentation["object"]
        new_test_data.append(original_test)
    

In [36]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/test_aug.jsonl")


In [7]:
new_test_data[29]

{'example_type': 'recent',
 'edit': {'prompt': 'The date of birth of Gordiya is 0550-01-01T00:00:00Z.',
  'subject_id': 'Q5950714',
  'relation': 'DATE_OF_BIRTH',
  'target_id': '0550-01-01T00:00:00Z',
  'subject': 'Date of birth of Gordiya',
  'object': '0550-01-01T00:00:00Z'},
 'Logical_Generalization': [],
 'Compositionality_I': [],
 'Compositionality_II': [],
 'Subject_Aliasing': [{'test_queries': [{'prompt': 'The date of birth of Gurdiya is',
     'answers': [{'value': '0550-01-01T00:00:00Z', 'aliases': []}],
     'query_type': 'regular',
     'subject_id': 'Q5950714',
     'relation': 'DATE_OF_BIRTH',
     'target_ids': ['0550-01-01T00:00:00Z'],
     'phrase': 'The date of birth of Gurdiya is'}],
   'test_condition': 'OR',
   'condition_queries': [{'prompt': 'The date of birth of Gurdiya is',
     'answers': [{'value': '0550-01-01T00:00:00Z', 'aliases': []}],
     'query_type': 'regular',
     'subject_id': 'Q5950714',
     'relation': 'DATE_OF_BIRTH',
     'target_ids': ['0550-0